# Data Proposal

So I've settled on a project regarding the connection between agriculture commodity prices and their futures trading volumes. I will also try to figure out whether these are somehow tied to global data.

I think the data that I need for this project is relatively straightforward: price data, trading volume data, and some nominal measure of global temperature.

Since my project will essentailly compare the market's reaction to changing prices, I think I'll just have to focus on finding market data.

In [1]:
import pandas as pd 

## Finding global temperatures

This should be easier, I think. I've found this [cool website](https://datahub.io) that has a couple datasets. I think they find interesting datasets and clean it up a little bit for other people to use.

There's a couple temperature-related datasets, but I think I'll use [this one](https://datahub.io/core/global-temp). It compares current temperatures to a historical benchmark, which sounds useful when seeing how temperatures have changed over time.

In [2]:
# gonna just try to clean up this data i guess

url = "https://pkgstore.datahub.io/core/global-temp/monthly_csv/data/5c846179d4938961e3f7515a26bf9976/monthly_csv.csv"
raw_temp = pd.read_csv(url)
raw_temp.head()

,Source,Date,Mean
0,GCAG,2016-12-15,0.7895
1,GISTEMP,2016-12-15,0.8100
2,GCAG,2016-11-15,0.7504
3,GISTEMP,2016-11-15,0.9300
4,GCAG,2016-10-15,0.7292


In [3]:
# just need to clean this up a little bit — I'll probably explain what each step is in the final project

grouped_data = raw_temp.groupby("Source")
temp = grouped_data.get_group("GISTEMP")
temp = temp.drop(temp.index[144:])
temp = temp.reindex(index = temp.index[::-1])
temp = temp.drop("Source", axis = 1)
temp = temp.rename(columns = {"Mean": "MeanTemperatureDeviation"})
df = temp
df = df.reset_index()
df = df.drop("index", axis = 1)
df = df.reset_index()

## Futures trading volume data

The only free public information I could find on futures trading volume is from _[Intercontinental Exchange](https://www.theice.com/marketdata/reports/8)_. It's a pain in the ass, but I pulled the data by hand and cleaned it. However, the data only goes back to 2005, and thus that's the timeframe of data I'll be using.

In [4]:
# reading in the data and cutting it down to the right size
url2 = "https://raw.githubusercontent.com/yorktruewang/data-bootcamp/master/Final%20Project/Commodity%20Futures%20Volumes.csv"
f_v = pd.read_csv(url2)
f_v.head()
f_v = f_v.rename(columns = {"Unnamed: 0": "Date"})
f_v.tail()
f_v = f_v.drop(f_v.index[144:])
f_v = f_v.drop(axis = 1, columns = "PULP")

In [5]:
# let's make a big giant dataframe for the important data
coffee = f_v["COFFEE"]
df["coffee"] = coffee

sugar = f_v["SUGAR"]
df["sugar"] = sugar

cotton = f_v["COTTON"]
df["cotton"] = cotton

fcoj = f_v["FCOJ"]
df["fcoj"] = fcoj

cocoa = f_v["COCOA"]
df["cocoa"] = cocoa

mg = f_v["METALS & GRAINS"]
df["grains proxy"] = mg

## Commodity Prices

This one was much easier to find!

I feel like since the clients of exchanges are global and since agriculture is a global industry, we can use global data. As such, I've found global prices for agriculture commodities, as opposed to US-focused or something.

The data is from the IMF – cleaned up and found on FRED. The csv files all have the same structure (thank god) and I've uploaded the data to my github

In [6]:
WHEAT ="https://raw.githubusercontent.com/yorktruewang/data-bootcamp/master/Final%20Project/WHEAT%20INDEX.csv"
COCOA = "https://raw.githubusercontent.com/yorktruewang/data-bootcamp/master/Final%20Project/COCOA%20INDEX.csv"
SUGAR = "https://raw.githubusercontent.com/yorktruewang/data-bootcamp/master/Final%20Project/SUGAR%20INDEX.csv"
COTTON = "https://raw.githubusercontent.com/yorktruewang/data-bootcamp/master/Final%20Project/COTTON%20INDEX.csv"

def clean(url, name):
    placeholder = pd.read_csv(url)
    placeholder.columns = ["Date", name]    
    placeholder = placeholder.drop(placeholder.index[0:300])    
    placeholder = placeholder.drop(placeholder.index[144:449])    
    placeholder = placeholder.reset_index()
    placeholder = placeholder.drop("index", axis = 1)    
    df[name] = placeholder[name]

# Manually clean the price indices
clean(SUGAR, "sugarindex")
clean(COTTON, "cottonindex")
clean(COCOA, "cocoaindex")

# Manually create new dataframes for each commodity
sugardf = pd.DataFrame({"sugar":df["sugar"], "sugarindex": df["sugarindex"]})
cottondf = pd.DataFrame({"cotton":df["cotton"], "cottonindex": df["cottonindex"]})
cocoadf = pd.DataFrame({"cocoa":df["cocoa"], "cocoaindex": df["cocoaindex"]})

Okay so I have the data I need! It seems quite simple, but there's really a couple dimensions for this data: prices and trading volumes. There's no geographic location or anything, which really limits the types of analyses I can do on this topic. Regardless, I think it's quite interesting, and I'd like to look further into it.